In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp_api
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
import os
import json
import datetime
import dill

In [2]:
from tensorflow.keras.layers import (
    Dense, 
    Dropout,
    LSTMCell,
    RNN
)

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [77]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/" + timestamp
version_dir = "version/" + timestamp 

os.makedirs(log_dir)
os.makedirs(version_dir)
timestamp

'20200918-233033'

In [4]:
dataset_name = "NU_AR"

In [5]:
static_params = {
    'PAST_HISTORY': 16,
    'FUTURE_TARGET': 8,
    'BATCH_SIZE': 1024,
    'BUFFER_SIZE': 200000,
    'EPOCHS': 500,
 }

In [20]:
with open("static/pipeline.pkl", "rb") as p:
    pipeline = dill.load(p)

static_params["VOCAB_SIZE"] = pipeline["sparse_category_encoder"].vocab_size
static_params

{'PAST_HISTORY': 16,
 'FUTURE_TARGET': 8,
 'BATCH_SIZE': 1024,
 'BUFFER_SIZE': 200000,
 'EPOCHS': 500,
 'VOCAB_SIZE': 6}

In [7]:
hparams_simple = {
    "HP_LSTM_1_UNITS" : 128,
    "HP_LSTM_1_DROPOUT" : 0.5,
    "HP_LEARNING_RATE" : 1e-3,
}

In [8]:
hparams_multiple = {
    "HP_LSTM_1_UNITS" : 32,
    "HP_LSTM_2_UNITS" : 32,
    "HP_LSTM_1_DROPOUT" : 0.0,
    "HP_LSTM_2_DROPOUT" : 0.0,
    "HP_LEARNING_RATE" : 1e-3,
}

In [17]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 5)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 5)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [10]:
from sklearn.preprocessing import OneHotEncoder

In [64]:
train_set = np.genfromtxt("data/{}_train_set.csv".format(dataset_name), delimiter="\n", dtype=np.int32)
encoder = OneHotEncoder(dtype=np.float32)
onehot_train_set = encoder.fit_transform(train_set.reshape(-1, 1))
x_train, y_train = generate_timeseries(onehot_train_set.toarray(), 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
train_data = tf.data.Dataset.from_tensor_slices((x_train[:-1], y_train[:-1]))   # As 5 (= -1) is artificially injected at the end
train_data = train_data.cache().batch(static_params["BATCH_SIZE"]).shuffle(static_params["BUFFER_SIZE"])

In [65]:
val_set = np.genfromtxt("data/{}_val_set.csv".format(dataset_name), delimiter="\n", dtype=np.int32)
onehot_val_set = encoder.transform(val_set.reshape(-1, 1))
x_val, y_val = generate_timeseries(onehot_val_set.toarray(), 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.cache().batch(static_params["BATCH_SIZE"])

In [66]:
test_set = np.genfromtxt("data/{}_test_set_original.csv".format(dataset_name), delimiter="\n", dtype=np.int64)
processed_test_set = pipeline.transform(test_set.copy())
onehot_test_set = encoder.transform(processed_test_set.reshape(-1, 1))
x_test, y_test = generate_timeseries(onehot_test_set.toarray(), 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])

In [67]:
class ARSimple(keras.Model):
    def __init__(self, units, dropout, output_steps, output_size):
        super().__init__()
        self.output_steps = output_steps
        self.units = units
        self.lstm_cell = LSTMCell(units, dropout=dropout)

        self.lstm_rnn = RNN(self.lstm_cell, return_state=True)
        self.dense = Dense(output_size, activation="softmax")

    @tf.function
    def warmup(self, inputs):
        #onehot_inputs = tf.squeeze(tf.one_hot(inputs, static_params["VOCAB_SIZE"]), axis=2)

        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x, *state = self.lstm_rnn(inputs)

        # predictions.shape => (batch, features)
        prediction = self.dense(x)

        return prediction, state

    @tf.function
    def call(self, inputs, training=None):
        # Use a TensorArray to capture dynamically unrolled outputs.
        #predictions = []
        predictions = tf.TensorArray(tf.float32, size=self.output_steps, clear_after_read=False)
        # Initialize the lstm state
        prediction, state = self.warmup(inputs)

        # Insert the first prediction
        #predictions.append(prediction)
        predictions = predictions.write(0, prediction)

        # Run the rest of the prediction steps
        for i in tf.range(1, self.output_steps):
            # Use the last prediction as input.
            x = prediction

            # Execute one lstm step.
            x, state = self.lstm_cell(x, states=state, training=training)

            # Convert the lstm output to a prediction.
            prediction = self.dense(x)

            # Add the prediction to the output
            #predictions.append(prediction)
            predictions = predictions.write(i, prediction)

        # predictions.shape => (time, batch, features)
        #predictions = tf.stack(predictions)
        predictions = predictions.stack()

        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])

        return predictions

In [68]:
class ARMultiple(keras.Model):
    def __init__(self, units_1, units_2, dropout_1, dropout_2, output_steps, output_size):
        super().__init__()
        self.output_steps = output_steps
        self.units_1 = units_1
        self.units_2 = units_2
        self.dropout_1 = dropout_1
        self.dropout_2 = dropout_2

        self.lstm_cell_1 = LSTMCell(units_1, dropout=dropout_1)
        self.lstm_cell_2 = LSTMCell(units_2, dropout=dropout_2)

        self.lstm_rnn_1 = RNN(self.lstm_cell_1, return_state=True, return_sequences=True)
        self.lstm_rnn_2 = RNN(self.lstm_cell_2, return_state=True)
        self.dense = Dense(output_size, activation="softmax")

    @tf.function#(input_signature=[tf.TensorSpec(shape=[None, None, 1], dtype=tf.int32)])
    def warmup(self, inputs):
        onehot_inputs = tf.squeeze(tf.one_hot(inputs, static_params["VOCAB_SIZE"]), axis=2)

        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x_1, *state_1 = self.lstm_rnn_1(onehot_inputs)
        x_2, *state_2 = self.lstm_rnn_2(x_1)

        # predictions.shape => (batch, features)
        prediction = self.dense(x_2)

        return prediction, state_1, state_2

    @tf.function#(input_signature=[tf.TensorSpec(shape=[None, None, 1], dtype=tf.int32)])
    def call(self, inputs, training=None):
        # Use a TensorArray to capture dynamically unrolled outputs.
        #predictions = []
        predictions = tf.TensorArray(tf.float32, size=self.output_steps, clear_after_read=False)

        # Initialize the lstm state
        prediction, state_1, state_2 = self.warmup(inputs)

        # Insert the first prediction
        #predictions.append(prediction)
        predictions = predictions.write(0, prediction)

        # Run the rest of the prediction steps
        for i in tf.range(1, self.output_steps):
            # Use the last prediction as input.
            x = prediction

            # Execute one lstm step.
            x_1, state_1 = self.lstm_cell_1(x, states=state_1, training=training)
            x_2, state_2 = self.lstm_cell_2(x_1, states=state_2, training=training)

            # Convert the lstm output to a prediction.
            prediction = self.dense(x_2)

            # Add the prediction to the output
            #predictions.append(prediction)
            predictions = predictions.write(i, prediction)

        # predictions.shape => (time, batch, features)
        #predictions = tf.stack(predictions)
        predictions = predictions.stack()

        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])

        return predictions

In [74]:
model = ARSimple(
    units=hparams_simple["HP_LSTM_1_UNITS"], dropout=hparams_simple["HP_LSTM_1_DROPOUT"], 
    output_steps=static_params["FUTURE_TARGET"], output_size=static_params["VOCAB_SIZE"])

model = ARMultiple(
    units_1=hparams_multiple["HP_LSTM_1_UNITS"], units_2=hparams_multiple["HP_LSTM_2_UNITS"], dropout_1=hparams_multiple["HP_LSTM_1_DROPOUT"], 
    dropout_2=hparams_multiple["HP_LSTM_2_DROPOUT"], output_steps=static_params["FUTURE_TARGET"], output_size=static_params["VOCAB_SIZE"])

In [75]:
model.compile(
    optimizer=keras.optimizers.Nadam(hparams_simple["HP_LEARNING_RATE"]),
    loss=tfa.losses.SigmoidFocalCrossEntropy(),
    #loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

In [78]:
with tf.summary.create_file_writer(log_dir).as_default():
    hp_api.hparams(hparams_simple)
    history = model.fit(train_data, validation_data=val_data, epochs=50, callbacks=[
        keras.callbacks.EarlyStopping('val_loss', patience=5),
        keras.callbacks.TensorBoard(log_dir)
        ])

    loss, acc = model.evaluate(x_test, y_test)
    tf.summary.scalar('test_loss', loss, step=x_test.shape[0])
    tf.summary.scalar('test_accuracy', acc, step=x_test.shape[0])

Epoch 1/50
7/7 [==============================] - 0s 49ms/step - loss: 0.0204 - accuracy: 0.9741 - val_loss: 0.0043 - val_accuracy: 0.9994
Epoch 2/50
7/7 [==============================] - 0s 43ms/step - loss: 0.0200 - accuracy: 0.9742 - val_loss: 0.0056 - val_accuracy: 0.9994
Epoch 3/50
7/7 [==============================] - 0s 45ms/step - loss: 0.0198 - accuracy: 0.9742 - val_loss: 0.0065 - val_accuracy: 0.9994
Epoch 4/50
7/7 [==============================] - 0s 41ms/step - loss: 0.0199 - accuracy: 0.9742 - val_loss: 0.0047 - val_accuracy: 0.9994
Epoch 5/50
7/7 [==============================] - 0s 41ms/step - loss: 0.0192 - accuracy: 0.9743 - val_loss: 0.0056 - val_accuracy: 0.9994
Epoch 6/50
7/7 [==============================] - 0s 41ms/step - loss: 0.0188 - accuracy: 0.9746 - val_loss: 0.0035 - val_accuracy: 0.9994
Epoch 7/50
7/7 [==============================] - 0s 39ms/step - loss: 0.0187 - accuracy: 0.9749 - val_loss: 0.0059 - val_accuracy: 0.9994
Epoch 8/50
7/7 [===========

In [21]:
model.summary()

Model: "ar_simple"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
lstm_cell (LSTMCell)         multiple                  4992
_________________________________________________________________
rnn (RNN)                    multiple                  4992
_________________________________________________________________
dense (Dense)                multiple                  198
Total params: 5,190
Trainable params: 5,190
Non-trainable params: 0
_________________________________________________________________


In [22]:
tf.saved_model.save(model, version_dir, 
    signatures=model.call.get_concrete_function(tf.TensorSpec(shape=[None, None, 1], dtype=tf.int32, name="call")))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: version/20200918-205457\assets


In [ ]:
with open("version/{}/evaluate.csv".format(timestamp), "w") as r:
    r.write("loss, accuracy\n")
    r.write("{}, {}".format(loss, acc))

new_model = tf.saved_model.load("version/" + "20200918-210348")

inference = new_model.signatures["serving_default"]

inference(tf.constant(x_train[0].reshape(1, -1, 1)))